In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tqdm
import random

from experiments.npe.model import get_npe_model, get_npe_model_replicate

In [2]:
data = np.load('data/threecircles_1.0.npz')

In [3]:
data.files

['train_y',
 'val_y',
 'arr_0',
 'arr_1',
 'arr_2',
 'arr_3',
 'arr_4',
 'arr_5',
 'arr_6',
 'arr_7',
 'arr_8',
 'arr_9']

In [6]:
train_x = []
val_x = []

for i in range(5):
    train_x.append(data['arr_%i' % (i)])
    val_x.append(data['arr_%i' % (i + 5)])
    
train_y = data['train_y']
val_y = data['val_y']

In [5]:
train_x[4] = np.ones((3000000, 1))
train_x[5] = np.ones((3000000, 1))
train_x[6] = np.ones((3000000, 1))

IndexError: list assignment index out of range

In [ ]:
def breakdown(X):
    return [np.array([x[i] for x in X]) for i in range(len(X[0]))]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interpn

def density_scatter( x , y, ax = None, sort = True, bins = 20, **kwargs )   :
    """
    Scatter plot colored by 2d histogram
    """
    if ax is None :
        fig , ax = plt.subplots()
    data , x_e, y_e = np.histogram2d( x, y, bins = bins)
    z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False )

    # Sort the points by density, so that the densest points are plotted last
    if sort :
        idx = z.argsort()
        x, y, z = x[idx], y[idx], z[idx]

    ax.scatter( x, y, c=z, **kwargs )
    return ax

In [ ]:
px = []
py = []

for t in range(100000):
    px.append(train_x[0][t][4] + (train_x[0][t][6] + train_y[t][0]) / 10.0)
    py.append(train_x[0][t][5] + (train_x[0][t][7] + train_y[t][1]) / 10.0)
    
    # Velocity
#     px.append(points_n[0][0] - points_p[0][0])
#     py.append(points_n[0][1] - points_p[0][1])
    
#     px.append(points_p[0][0])
#     py.append(points_p[0][1])
    
density_scatter(np.array(px), np.array(py), bins=10)

In [ ]:
train_x[0][:, 1].max()

In [ ]:
model = get_npe_model(max_pairs = 2, state_dim = 4, hidden_size = 256, variational = True)

opt = tf.keras.optimizers.RMSprop(learning_rate=0.0003, clipnorm=1.0)
model.compile(loss=lambda y, d: -d.log_prob(y), optimizer=opt, metrics=['mse'])

In [14]:
def get_lossfunc(z_mux, z_muy, z_sx, z_sy, z_corr, x_data, y_data):
    """
    Function to calculate given a 2D distribution over x and y, and target data
    of observed x and y points
    params:
    z_mux : mean of the distribution in x
    z_muy : mean of the distribution in y
    z_sx : std dev of the distribution in x
    z_sy : std dev of the distribution in y
    z_rho : Correlation factor of the distribution
    x_data : target x points
    y_data : target y points
    """
    step = tf.constant(1e-3, dtype=tf.float32, shape=(1, 1))

    # Calculate the PDF of the data w.r.t to the distribution
    result0_1 = tf_2d_normal(x_data, y_data, z_mux, z_muy, z_sx, z_sy, z_corr)
    result0_2 = tf_2d_normal(
        tf.add(x_data, step), y_data, z_mux, z_muy, z_sx, z_sy, z_corr
    )
    result0_3 = tf_2d_normal(
        x_data, tf.add(y_data, step), z_mux, z_muy, z_sx, z_sy, z_corr
    )
    result0_4 = tf_2d_normal(
        tf.add(x_data, step), tf.add(y_data, step), z_mux, z_muy, z_sx, z_sy, z_corr
    )

    result0 = tf.divide(
        tf.add(tf.add(tf.add(result0_1, result0_2), result0_3), result0_4),
        tf.constant(4.0, dtype=tf.float32, shape=(1, 1)),
    )
    result0 = tf.multiply(tf.multiply(result0, step), step)

    # For numerical stability purposes
    epsilon = 1e-20

    # Apply the log operation
    result1 = -tf.math.log(tf.maximum(result0, epsilon))  # Numerical stability

    # Sum up all log probabilities for each data point
    return tf.reduce_sum(result1)


def tf_2d_normal(x, y, mux, muy, sx, sy, rho):
    """
        Function that implements the PDF of a 2D normal distribution
        params:
        x : input x points
        y : input y points
        mux : mean of the distribution in x
        muy : mean of the distribution in y
        sx : std dev of the distribution in x
        sy : std dev of the distribution in y
        rho : Correlation factor of the distribution
        """
    # eq 3 in the paper
    # and eq 24 & 25 in Graves (2013)
    # Calculate (x - mux) and (y-muy)
    normx = tf.subtract(x, mux)
    normy = tf.subtract(y, muy)
    # Calculate sx*sy
    sxsy = tf.multiply(sx, sy)
    # Calculate the exponential factor
    z = (
        tf.square(tf.divide(normx, sx))
        + tf.square(tf.divide(normy, sy))
        - 2 * tf.divide(tf.multiply(rho, tf.multiply(normx, normy)), sxsy)
    )
    negRho = 1 - tf.square(rho)
    # Numerator
    result = tf.exp(tf.divide(-z, 2 * negRho))
    # Normalization constant
    denom = 2 * np.pi * tf.multiply(sxsy, tf.sqrt(negRho))
    # Final PDF calculation
    result = tf.divide(result, denom)
    return result


def get_coef(output):
    # eq 20 -> 22 of Graves (2013)

    z = output
    
    # Split the output into 5 parts corresponding to means, std devs and corr
    z_mux, z_muy, z_sx, z_sy, z_corr = tf.split(z, 5, 1)

    # The output must be exponentiated for the std devs
    z_sx = tf.exp(z_sx)
    z_sy = tf.exp(z_sy)
    # Tanh applied to keep it in the range [-1, 1]
    z_corr = tf.tanh(z_corr)

    return [z_mux, z_muy, z_sx, z_sy, z_corr]


def sample_gaussian_2d(mux, muy, sx, sy, rho):
    """
    Function to sample a point from a given 2D normal distribution
    params:
    mux : mean of the distribution in x
    muy : mean of the distribution in y
    sx : std dev of the distribution in x
    sy : std dev of the distribution in y
    rho : Correlation factor of the distribution
    """
    # Extract mean
    mean = [mux, muy]
    # Extract covariance matrix
    cov = [[sx * sx, rho * sx * sy], [rho * sx * sy, sy * sy]]
    # Sample a point from the multivariate normal distribution
    x = np.random.multivariate_normal(mean, cov, 1)
    return x[0][0], x[0][1]

In [52]:
tf.keras.backend.set_floatx('float64')

model = get_npe_model(max_pairs = 2, state_dim = 4, hidden_size = 50, variational = True)

def lossfunc(ytrue, ypred):
    z_mux, z_muy, z_sx, z_sy, z_corr = get_coef(ypred)
    return get_lossfunc(z_mux, z_muy, z_sx, z_sy, z_corr, ytrue[...,0], ytrue[...,1])

def gaussian_nll(ytrue, ypreds):
    """Keras implmementation og multivariate Gaussian negative loglikelihood loss function. 
    This implementation implies diagonal covariance matrix.
    
    Parameters
    ----------
    ytrue: tf.tensor of shape [n_samples, n_dims]
        ground truth values
    ypreds: tf.tensor of shape [n_samples, n_dims*2]
        predicted mu and logsigma values (e.g. by your neural network)
        
    Returns
    -------
    neg_log_likelihood: float
        negative loglikelihood averaged over samples
        
    This loss can then be used as a target loss for any keras model, e.g.:
        model.compile(loss=gaussian_nll, optimizer='Adam') 
    
    """
    
    n_dims = int(int(ypreds.shape[1])/2)
    mu = ypreds[:, 0:n_dims]
    logsigma = ypreds[:, n_dims:]

    mse = -0.5*tf.keras.backend.sum(tf.keras.backend.square((ytrue-mu)/tf.keras.backend.exp(logsigma)),axis=1)
    sigma_trace = -tf.keras.backend.sum(logsigma, axis=1)
    log2pi = -0.5*n_dims*np.log(2*np.pi)

    log_likelihood = mse+sigma_trace+log2pi

    return tf.keras.backend.mean(-log_likelihood)

def mse2(ytrue, ypred):
    return ((ytrue[...,0] - ypred[...,0])**2 + (ytrue[...,1] - ypred[...,1])**2) / 2

def mse(ytrue, ypred):
    z_mux, z_muy, z_sx, z_sy, z_corr = get_coef(ypred)
    return ((ytrue[...,0] - z_mux)**2 + (ytrue[...,1] - z_muy)**2) / 2

opt = tf.keras.optimizers.RMSprop(learning_rate=0.003)
model.compile(loss='mse', optimizer=opt)

In [53]:
history = model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=20, batch_size=2048, shuffle=True)

Train on 3000000 samples, validate on 300000 samples
Epoch 1/20
3000000/3000000 [==============================] - 33s 11us/sample - loss: 0.0810 - val_loss: 0.0864
Epoch 2/20
3000000/3000000 [==============================] - 32s 11us/sample - loss: 0.0619 - val_loss: 0.0490
Epoch 3/20
3000000/3000000 [==============================] - 32s 11us/sample - loss: 0.0347 - val_loss: 0.0355
Epoch 4/20
3000000/3000000 [==============================] - 32s 11us/sample - loss: 0.0280 - val_loss: 0.0287
Epoch 5/20
3000000/3000000 [==============================] - 32s 11us/sample - loss: 0.0253 - val_loss: 0.0238
Epoch 6/20
3000000/3000000 [==============================] - 32s 11us/sample - loss: 0.0234 - val_loss: 0.0303
Epoch 7/20
3000000/3000000 [==============================] - 32s 11us/sample - loss: 0.0221 - val_loss: 0.0192
Epoch 8/20
3000000/3000000 [==============================] - 32s 11us/sample - loss: 0.0210 - val_loss: 0.0187
Epoch 9/20
3000000/3000000 [=======================

In [56]:
p = model(val_x)
p[0].numpy()

array([ 0.23323942, -0.21783152])

In [ ]:
val_y[18]

In [ ]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0003)
model.compile(loss='mse', optimizer=opt)

In [63]:
# model = tf.keras.models.load_model('model_zoo/npe_replicate_b1024_e50.h5')
from experiments.npe.simulate import show_simulation, show_simulation_variational
show_simulation(model)
# show_simulation_variational(model, length=15)

KeyboardInterrupt: 

In [64]:
model.save('model_zoo/stochastic_npe_1.h5')

In [ ]:
len(train_x)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
def get_velocity_mse():
    p = model.predict(val_x)
    return (((np.vstack(val_x[:, 0])[:, 6:] + p) - (np.vstack(val_x[:, 0])[:, 6:] + val_y))**2).mean()

In [ ]:
get_velocity_mse()

In [ ]:
p[2]

In [ ]:
val_x[2]

In [ ]:
val_y[2]

In [ ]:
b_train_x[0]

In [ ]:
model.save('model_zoo/npe_replicate_b1024_e50.h5')

In [ ]:
model = tf.keras.models.load_model('model_zoo/npe4.h5')

In [ ]:
p = model.predict(b_val_x)

In [ ]:
p[:, 0].min()

In [ ]:
for x in np.array(train_y):
    print(x)

In [ ]:
model.layers[-3]([train_x[0][41:42], train_x[2][41:42], np.array([[1.0]])])

In [ ]:
train_x[4].argmax()